# Data Cleaning and Preparation

- S significant amount of time is spent on data preparation: loading, cleaning, transforming, andarranging. 

- Such tasks are often reported to take up 80% or more of an analyst's time. 

- Pandas provides you with a high-level, flexible, and fast set of tools to enable you to manipulate data into the right form.

- This chapter discusses tools for missing data, duplicate data, string manipulation, and some other analytical data transformations. 

In [ ]:
import numpy as np
import pandas as pd

PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt

plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## Handling Missing Data

> For data with float64 dtype, pandas uses the floating-point value NaN (Not a Number) to represent missing data. We call this a sentinel value: when present, it indicates a missing (or null) value:

In [ ]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data

The isna method gives us a boolean Series with True where values are null:

In [ ]:
string_data.isnull()

In pandas, we've adopted a convention used in the R programming language by referring to missing data as NA, which stands for not available

In [ ]:
#The built-in Python None value is also treated as NA:
string_data = pd.Series(["aardvark", np.nan, None, "avocado"])

string_data


In [ ]:
float_data = pd.Series([1, 2, None], dtype='float64')
float_data

### Filtering Out Missing Data

- There are a few ways to filter out missing data.

- While you always have the option to do it by hand using pandas.isna and boolean indexing, the dropna can be helpful. 

- On a Series, it returns the Series with only the non-null data and index values

In [ ]:
from numpy import nan as NA

data = pd.Series([1, NA, 3.5, NA, 7])
data

In [ ]:
data.dropna()

In [ ]:
data[data.notnull()] #This is the same things as above

- There are different ways you may need to remove missing data. 

- You may want to drop rows or columns that are all NA or only those rows or columns containing any NAs at all. 

- dropna by default drops any row containing a missing value:

In [ ]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                     [NA, NA, NA], [NA, 6.5, 3.]])
data

> Passing how="all" will only drop rows that are all NA:

In [ ]:
data.dropna(how='all')

These functions return new objects by default and do not modify the contents of the original object.

In [ ]:
data[4] = NA
data

To drop columns in the same way, pass axis="columns":

In [ ]:
data.dropna(axis=1, how='all')

Using threshold, you can specify the minimum number of non-null values required to keep a column:

In [ ]:
df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df


In [ ]:
df.dropna()

Suppose you want to keep only rows containing at most a certain number of missing observations. You can indicate this with the thresh argument:

In [ ]:
df.dropna(thresh=2)

### Filling In Missing Data

- Rather than filtering out missing data (and potentially discarding other data along with it), 

- You may want to fill in the “holes” in any number of ways. 
  
- For most purposes, the fillna method is the workhorse function to use. Calling fillna with a constant replaces missing values with that value:

In [ ]:
df

In [ ]:
df.fillna(0)

Calling fillna with a dictionary, you can use a different fill value for each column:

In [ ]:
df.fillna({1: 0.5, 2: 0}) 

In [ ]:
df

In [ ]:
df.fillna({1: 0.5, 2: 0}) 

In [ ]:
df

In [ ]:
df_new = df.fillna({1: 0.5, 2: 0}) 
df_new

>  fill in-place. Note: this will modify any other views on this object (e.g., a no-copy slice for a column in a DataFrame).


In [ ]:
df.fillna(0, inplace=True)

In [ ]:
df

The same interpolation methods available for reindexing can be used with fillna:

In [ ]:
df = pd.DataFrame(np.random.standard_normal((6, 3)))
df

In [ ]:
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df


In [ ]:
df.fillna(method='bfill')


With fillna you can do lots of other things such as simple data imputation using the median or mean statistics:

In [ ]:
data = pd.Series([1., NA, 3.5, NA, 7])
data

In [ ]:
data.mean()

In [ ]:
data.fillna(data.mean())

## Data Transformation

### Removing Duplicates

Duplicate rows may be found in a DataFrame for any number of reasons. Here is an example:

In [ ]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

In [ ]:
data.duplicated()

In [ ]:
data.drop_duplicates()

- Both of these methods by default consider all of the columns; alternatively, you can specify any subset of them to detect duplicates. 

- Suppose we had an additional column of values and wanted to filter duplicates only based on the "k1" column:

In [ ]:
data['v1'] = range(7)


In [ ]:
data

To remove duplicates on specific column(s), use subset.



In [ ]:
data.drop_duplicates(subset=["k1", "k1", "v1"])

Relatedly, drop_duplicates returns a DataFrame with rows where the duplicated array is False filtered out:

In [ ]:
data.drop_duplicates(['k1', 'k2'], keep='last')

### Transforming Data Using a Function or Mapping

- For many datasets, you may wish to perform some transformation based on the values in an array, Series, or column in a DataFrame. 

- Consider the following hypothetical data collected about various kinds of meat:

In [ ]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

Suppose you wanted to add a column indicating the type of animal that each food came from. Let’s write down a mapping of each distinct meat type to the kind of animal:

In [ ]:
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

In [ ]:
data["food"]

The map method on a Series accepts a function or dictionary-like object containing a mapping to do the transformation of values:

In [ ]:
data["animal"] = data["food"].map(meat_to_animal)

In [ ]:
data

Using map is a convenient way to perform element-wise transformations and other data cleaning–related operations.

In [ ]:
lowercased = data['food'].str.lower()
lowercased


### Replacing Values

- Filling in missing data with the fillna method is a special case of more general value replacement. 

- As you've already seen, map can be used to modify a subset of values in an object but replace provides a simpler and more flexible way to do so. Let’s consider this Series:

In [ ]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

The -999 values might be sentinel values for missing data. To replace these with NA values that pandas understands, we can use replace, producing a new Series:

In [ ]:
data.replace(-999, np.nan)

If you want to replace multiple values at once, you instead pass a list and then the substitute value:

In [ ]:
data.replace([-999, -1000], np.nan)

To use a different replacement for each value, pass a list of substitutes:

In [ ]:
data.replace([-999, -1000], [np.nan, 0])

In [ ]:
data

In [ ]:
data.where(data > 0, 3)

The argument passed can also be a dictionary:

In [ ]:
data.replace({-999: np.nan, -1000: 0})

### Renaming Axis Indexes

Like values in a Series, axis labels can be similarly transformed by a function or mapping of some form to produce new, differently labeled objects. 

You can also modify the axes in-place without creating a new data structure. Here’s a simple example:

In [ ]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])

In [ ]:
data

In [ ]:
transform = lambda x: x[:4].upper()


In [ ]:
transform("baco")

In [ ]:
data.index

In [ ]:
transform = lambda x: x[:4].upper()
data.index.map(transform)

You can assign to the index attribute, modifying the DataFrame in-place:

In [ ]:
data.index = data.index.map(transform)
data

If you want to create a transformed version of a dataset without modifying the original, a useful method is rename:

In [ ]:
data.rename(index=str.title, columns=str.upper)

Notably, rename can be used in conjunction with a dictionary-like object providing new values for a subset of the axis labels:

In [ ]:
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'peekaboo'})

rename saves you from the chore of copying the DataFrame manually and assigning to its index and columns attributes.

In [ ]:
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

### Discretization and Binning

- Continuous data is often discretized or otherwise separated into “bins” for analysis.

- Suppose you have data about a group of people in a study, and you want to group them into discrete age buckets:

In [ ]:
import pandas as pd
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

Let’s divide these into bins of 18 to 25, 26 to 35, 36 to 60, and finally 61 and older. To do so, you have to use pandas.cut:

In [ ]:
bins = [18, 25, 35, 60, 100]
age_categories = pd.cut(ages, bins)
age_categories

- The object pandas returns is a special Categorical object. 

- The output you see describes the bins computed by pandas.cut. 

- Each bin is identified by a special (unique to pandas) interval value type containing the lower and upper limit of each bin:

In [ ]:
age_categories.codes

In [ ]:
age_categories.categories

In [ ]:
age_categories.categories[0]

In [ ]:
pd.value_counts(age_categories)

> pd.value_counts(categories) are the bin counts for the result of pandas.cut.

- In the string representation of an interval, a parenthesis means that the side is open (exclusive), while the square bracket means it is closed (inclusive). 

- You can change which side is closed by passing right=False:

In [ ]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

In [ ]:
pd.cut(ages, [18, 26, 36, 61, 100], right=True)

In [ ]:
pd.cut(ages, [18, 26, 36, 61, 100]) #Default is right=True

You can override the default interval-based bin labeling by passing a list or array to the labels option:

In [ ]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

If you pass an integer number of bins to pandas.cut instead of explicit bin edges, it will compute equal-length bins based on the minimum and maximum values in the data.

Consider the case of some uniformly distributed data chopped into fourths:

In [ ]:
data = np.random.rand(20)
pd.cut(data, 4, precision=2) # precision=2 option limits the decimal precision to two digits.

- A closely related function, pandas.qcut, bins the data based on sample quantiles. 

- Depending on the distribution of the data, using pandas.cut will not usually result in each bin having the same number of data points. 
  
- Since pandas.qcut uses sample quantiles instead, you will obtain roughly equally-sized bins:

In [ ]:
data = np.random.randn(1000)  # Normally distributed
cats = pd.qcut(data, 4)  # Cut into quartiles
cats
pd.value_counts(cats)

### Detecting and Filtering Outliers

- Filtering or transforming outliers is largely a matter of applying array operations. 

- Consider a DataFrame with some normally distributed data:

In [ ]:
data = pd.DataFrame(np.random.randn(1000, 4))
data

In [ ]:
data.describe()

Suppose you wanted to find values in one of the columns exceeding 3 in absolute value:

In [ ]:
data

In [ ]:
col = data[2]
col

In [ ]:
np.abs(col)>3

In [ ]:
col[np.abs(col) > 3]

To select all rows having a value exceeding 3 or –3, you can use the any method on a boolean DataFrame:

In [ ]:
data[(np.abs(data) > 3).any(axis="columns")]

> The parentheses around data.abs() > 3 are necessary in order to call the any method on the result of the comparison operation.

- Values can be set based on these criteria. Here is code to cap values outside the interval –3 to 3

In [ ]:
data[(np.abs(data) > 3).any(axis="columns")]

> Sign function: The sign function returns -1 if x < 0, 0 if x==0, 1 if x > 0. nan is returned for nan inputs.

 

In [ ]:
np.sign([-5., 4.5])

In [ ]:
np.sign(data)

In [ ]:
  data[np.abs(data) > 3] = np.sign(data) * 3

In [ ]:
data

In [ ]:
data

The statement np.sign(data) produces 1 and –1 values based on whether the values in data are positive or negative: Zxcfg

In [ ]:
np.sign(data).head()

### Permutation and Random Sampling

- Permutation is a random sampling technique that can be used to randomly select a subset of rows from a DataFrame.
  
- Permuting (randomly reordering) a Series or the rows in a DataFrame is possible using the numpy.random.permutation function.
 
- Calling permutation with the length of the axis you want to permute produces an array of integers indicating the new ordering:

In [ ]:
np.arange(5 * 4)

In [ ]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
df

In [ ]:
sampler = np.random.permutation(5)
sampler

- Return element in a given index in the array.

In [ ]:
df.take(sampler)

In [ ]:
df.iloc[sampler] # This is the same as df.take(sampler)

- To select a random subset without replacement (the same row cannot appear twice), you can use the sample method on Series and DataFrame:

In [ ]:
df.sample(n=3)

To generate a sample with replacement (to allow repeat choices), pass replace=True to sample:

In [ ]:
choices = pd.Series([5, 7, -1, 6, 4])
draws = choices.sample(n=10, replace=True)
draws

### Joining Dataframe

Lets join the two DataFrames on the "

In [ ]:
df1 = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3', 'K4', 'K5'],
                    'A': ['A0', 'A1', 'A2', 'A3', 'A4', 'A5']})

df2 = pd.DataFrame({'key': ['K0', 'K1', 'K2'],
                       'B': ['B0', 'B1', 'B2']})

df1.join(df2, lsuffix='_caller', rsuffix='_other')

If we want to join using the key columns, we need to set key to be the index in both df and other. The joined DataFrame will have key as its index.


In [ ]:
df1.set_index('key').join(df2.set_index('key'))

Another option to join using the key columns is to use the on parameter. DataFrame.join always uses other's index but we can use any column in df. This method preserves the original DataFrame's index in the result.



In [ ]:
df1.join(df2.set_index('key'), on='key')

Using non-unique key values shows how they are matched.



In [ ]:
df1 = pd.DataFrame({'key': ['K0', 'K1', 'K1', 'K3', 'K0', 'K1'],
                   'A': ['A0', 'A1', 'A2', 'A3', 'A4', 'A5']})

df2 = pd.DataFrame({'key': ['K0', 'K1', 'K2'],
                       'B': ['B0', 'B1', 'B2']})



In [ ]:
df1.join(df2.set_index('key'), on='key')

### Computing Indicator/Dummy Variables

- Another type of transformation for statistical modeling or machine learning applications is converting a categorical variable into a “dummy” or “indicator” matrix. 

- If a column in a DataFrame has k distinct values, you would derive a matrix or DataFrame with k columns containing all 1s and 0s. 

- pandas has a pandas.get_dummies function for doing this, though you could also devise one yourself. Let’s consider an example DataFrame:

In [ ]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})
df

In [ ]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})
df

In [ ]:
pd.get_dummies(df['key'])

In some cases, you may want to add a prefix to the columns in the indicator DataFrame, which can then be merged with the other data. pandas.get_dummies has a prefix argument for doing this:

In [ ]:
dummies = pd.get_dummies(df['key'], prefix='key')
dummies

In [ ]:
df

In [ ]:
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

A useful recipe for statistical applications is to combine pandas.get_dummies with a discretization function like pandas.cut:

In [ ]:
np.random.seed(12345)
values = np.random.rand(10)
values
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

## Extension Data Types

- Pandas was originally built upon the capabilities present in NumPy, an array computing library used primarily for working with numerical data. 

- Many pandas concepts, such as missing data, were implemented using what was available NumPy while trying to maximize compatibility between libraries that used NumPy and pandas together.

Building on NumPy led to a number of shortcomings, such as:

- Missing data handling for some numerical data types, such as integers and booleans, was incomplete. As a result, when missing data would be introduced into such data, pandas would convert the data type to float64 and use np.nan to represent null values. This had compounding effects by introducing subtle issues into many pandas algorithms.

- Data sets with a lot of string data were computationally expensive and used a lot of memory.

- Some data types, like time intervals, timedeltas, timestamps with time zones could not be supported efficiently without using computationally expensive arrays of Python objects

> More recently, pandas has developed an extension type system allowing for new data types to be added even if they are not supported natively by NumPy. These new data types could be treated as first class alongside data coming from NumPy arrays.

In [ ]:
s = pd.Series([1, 2, 3, None])
s

In [ ]:
s.dtype

> Mainly for backward compatibility reasons, Series uses the legacy behavior of using a float64 data type and np.nan for the missing value. We could create this Series instead using pandas.Int64Dtype:

In [ ]:
s = pd.Series([1, 2, 3, None], dtype=pd.Int64Dtype())
s

The output "NA" indicates that a value is missing for an extension type array. This uses the special pandas.NA sentinel value

In [ ]:
s[3] is pd.NA

We could also have used the shorthand "Int64" instead of pd.Int64Dtype() to specify the type. The capitalization is necessary otherwise you will be a NumPy-based non-extension type:

In [ ]:
s = pd.Series([1, 2, 3, None], dtype="Int64")
s

pandas also has an extension type specialized for string data that does not use NumPy object arrays (it requires the pyarrow library, which you may need to install separately):

In [ ]:
s = pd.Series(['one', 'two', None, 'three'], dtype=pd.StringDtype())
s

> These string arrays generally use much less memory and are frequently computationally more efficient for doing operations on large data sets.

Extension types can be passed to the Series astype method, allowing you to convert easily as part of your data cleaning process:

In [ ]:
df = pd.DataFrame({"A": [1, 2, None, 4],
   .....:                    "B": ["one", "two", "three", None],
   .....:                    "C": [False, None, False, True]})

df


In [ ]:
type(df)

In [ ]:
df["A"] = df["A"].astype("Int64")
df

In [ ]:
df["B"] = df["B"].astype("string")
df

In [ ]:
df["C"] = df["C"].astype("boolean")
df

## String Manipulation

- Python has long been a popular raw data manipulation language in part due to its ease of use for string and text processing. 

- Most text operations are made simple with the string object’s built-in methods. 

- For more complex pattern matching and text manipulations, regular expressions may be needed. 

- pandas adds to the mix by enabling you to apply string and regular expressions concisely on whole arrays of data, additionally handling the annoyance of missing data.

## Python Built-In String Object Methods

### String Object Methods

 
- comma-separated string can be broken into pieces with split:

In [ ]:
val = 'a,b,  guido'
val.split(',')

- split is often combined with strip to trim whitespace (including line breaks):

In [ ]:
pieces = [x.strip() for x in val.split(',')]
pieces

These substrings could be concatenated together with a two-colon delimiter using addition:

In [ ]:
first, second, third = pieces
first + '::' + second + '::' + third

But this isn’t a practical generic method. A faster and more Pythonic way is to pass a list or tuple to the join method on the string "::":

In [ ]:
'::'.join(pieces)

Other methods are concerned with locating substrings. Using Python’s in keyword is the best way to detect a substring, though index and find can also be used:

In [ ]:
val

In [ ]:
'guidos' in val


In [ ]:
val.index(',')


Note the difference between find and index is that index raises an exception if the string isn’t found (versus returning –1):

In [ ]:
val

In [ ]:
val.find(':')

In [ ]:
val.index(':')

Relatedly, count returns the number of occurrences of a particular substring:

In [ ]:
val.count(',')

replace will substitute occurrences of one pattern for another. It is commonly used to delete patterns, too, by passing an empty string:

In [ ]:
val.replace(',', '::')  
val.replace(',', '')

### Regular Expressions

- Regular expressions provide a flexible way to search or match (often more complex) string patterns in text. 

- A single expression, commonly called a regex, is a string formed according to the regular expression language. 

- Python’s built-in re module is responsible for applying regular expressions to strings; I’ll give a number of examples of its use here.

> The art of writing regular expressions could be a chapter of its own and thus is outside the book’s scope. 


- The re module functions fall into three categories: 
  
  - pattern matching, substitution, and splitting. 
  - Naturally these are all related; a regex describes a pattern to locate in the text, which can then be used for many purposes. 
  - Let’s look at a simple example: suppose we wanted to split a string with a variable number of whitespace characters (tabs, spaces, and newlines). 
  
  - The regex describing one or more whitespace characters is \s+:

In [ ]:
import re
text = "foo    bar\t baz  \tqux"
re.split('\s+', text)



> When you call re.split(r"\s+", text), the regular expression is first compiled, and then its split method is called on the passed text. You can compile the regex yourself with re.compile, forming a reusable regex object:

In [ ]:
regex = re.compile('\s+')
regex.split(text)

If, instead, you wanted to get a list of all patterns matching the regex, you can use the findall method:

In [ ]:
regex.findall(text)

> Read more about regular expressions in the Python documentation.

> Creating a regex object with re.compile is highly recommended if you intend to apply the same expression to many strings; doing so will save CPU cycles.

In [ ]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

# re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

In [ ]:
regex.findall(text)

In [ ]:
m = regex.search(text)
m
text[m.start():m.end()]

In [ ]:
print(regex.match(text))

In [ ]:
print(regex.sub('REDACTED', text))

In [ ]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags=re.IGNORECASE)

In [ ]:
m = regex.match('wesm@bright.net')
m.groups()

In [ ]:
regex.findall(text)

In [ ]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

###  String Functions in pandas

- Cleaning up a messy dataset for analysis often requires a lot of string manipulation. 

- To complicate matters, a column containing strings will sometimes have missing data:

In [ ]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = pd.Series(data)
data


> String and regular expression methods can be applied (passing a lambda or other function) to each value using data.map, but it will fail on the NA (null) values. To cope with this, Series has array-oriented methods for string operations that skip over and propagate NA values. These are accessed through Series’s str attribute; for example, we could check whether each email address has "gmail" in it with str.contains:

In [ ]:
data.str.contains('gmail')

- Note that the result of this operation has an object dtype. 

- pandas has extension types which provide for specialized treatment of strings, integers, and boolean data which until recently have had some rough edges when working with missing data:

In [ ]:
data_as_string_ext = data.astype('string')
data_as_string_ext


## Categorical Data

- This section introduces the pandas Categorical type
  
- Frequently, a column in a table may contain repeated instances of a smaller set of distinct values. We have already seen functions like unique and value_counts, which enable us to extract the distinct values from an array and compute their frequencies, respectively:

In [ ]:
values = pd.Series(['apple', 'orange', 'apple' , 'apple'] * 2)
values

In [ ]:
pd.unique(values)

In [ ]:
pd.value_counts(values)

> In data warehousing, a best practice is to use so-called dimension tables containing the distinct values and storing the primary observations as integer keys referencing the dimension table:

In [ ]:
values = pd.Series([0, 1, 0, 0] * 2)

values

In [ ]:
dim = pd.Series(['apple', 'orange'])
dim

In [ ]:
values

We can use the take method to restore the original Series of strings:

In [ ]:
dim.take(values)

## Conclusion